In [1]:
%reload_ext autoreload
%autoreload 2

"""
mc4を掃除して､fasttextでgood/badを分類して､クラスタリングして記録するサンプルコード
annotationもこのnotebookで行います

"""
from datasets import load_dataset


#mc4の読み込み
mc4_dataset = load_dataset('mc4', 'ja',split='train', streaming=True)

"""
#oscarなども読み込める
#ignore_verifications=Trueをつけないとエラーとなる
oscar_dataset = load_dataset('oscar', 'unshuffled_original_ja', 
                       split='train', 
                       ignore_verifications=True,
                       streaming=True)

"""

dataset=mc4_dataset

/Users/eijit/.asdf/installs/python/miniforge3-latest/envs/crawl/lib/python3.12/site-packages/datasets/load.py:1461: FutureWarning: The repository for mc4 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mc4
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
/Users/eijit/.cache/huggingface/modules/datasets_modules/datasets/mc4/78f7a2b7e2524fa44ee464ef429d011c365f5fe129283869e7fd76856aacb83a/mc4.py:284: FutureWarning: Dataset 'mc4' is deprecated and will be deleted. Use 'allenai/c4' instead.
  warnings.warn(


In [2]:
from src.cleaner.auto_cleaner import clean_text
from src.classifier.DatasetAnnotator import DatasetAnnotator

In [3]:
annotator=DatasetAnnotator(dataset,clean_func=clean_text,n_preload=50000)

In [4]:
#annotations
annotator.ask_annotations()


begin annotations
Enter: annnotate as "bad"
something: annotate as "good"
q: quit


In [ ]:
#fasttextで訓練
annotator.train_fasttext(autotuneDuration=120,wordNgrams=2)

In [ ]:
#テキストの教師なしクラスタリング: 結局使わない

#modelをannotations/text_labelsに保存しておく
#!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ja.300.bin.gz
#!gzip -d cc.ja.300.bin.gz

In [ ]:
from src.classifier.NounClustering import NounClustering
#clf=NounClustering()

#clf.train_wiki(n_samples=10**8) #wikipediaのタイトルで教師なしクラスタリングの学習

In [ ]:
#モデルの挙動確認
#カテゴリの分類がかなりいまいち｡
"""
for i in range(100):
    text=annotator.get_text_by_id(i)
    text_=text.replace("\n","")
    if text=="":continue
    is_noise=(annotator.predict(text))
    genre=clf.predict(text)
    print(is_noise,genre,text_[:300])
"""

In [ ]:
n_split=10**2
import os

corpus_dir="corpus/test"
if not os.path.exists(corpus_dir):
    os.makedirs(corpus_dir)

cnt=0
record_id=-1
for record in dataset:
    record_id+=1
    text=record['text']

    #テキストクリーン
    text=clean_text(text)
    if text=="":
        continue

    #記事の判定
    is_noise=annotator.predict(text)
    if is_noise==1:
        continue

    #ジャンルの判定
    #genre=clf.predict(text)
    cnt+=1

    d={
        "id":record_id,
        #"cat":genre,
        "text":text,
    }
    file_name=f"{corpus_dir}/{cnt//n_split}.txt"
    with open(file_name, "a") as f:
        f.write(str(d)+"\n")

    if cnt>1000:
        break

In [ ]:
#圧縮率
cnt/record_id